#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter ITP Audit Dashboard Parameters
Dashboard that shows performance metrics across browser to see the impact of ITP.
 1. Follow the steps in the below document
 1. https://docs.google.com/document/d/1HaRCMaBBEo0tSKwnofWNtaPjlW0ORcVHVwIRabct4fY/edit?usp=sharing
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  "dataset":"ITP_Audit_Dashboard", # BigQuery dataset for store dashboard tables.
  "sheet_url":"", # Sheet URL for the Segments sheet.
  "cm_account_id":"", # Campaign Manager Account Id.
  "advertiser_ids":"", # Comma separated list of Campaign Manager Advertiser Ids.  Leave blank for no advertiser filtering.
  "floodlight_configuration_id":, # Floodlight Configuration Id for the Campaign Manager floodlight report.
  "floodlight_report_name":"ITP_Audit_Dashboard_Floodlight", # Campaign Manager Floodlight report name.
  "dv360_report_name":"ITP_Audit_Browser_Report", # DV360 Browser report name.
  "cm_browser_report_name":"ITP_Audit_Dashboard_Browser", # Name of the Campaign Manager browser report.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute ITP Audit Dashboard
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [ { 'dataset': { 'auth': 'user',
                 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                         'description': 'BigQuery '
                                                        'dataset '
                                                        'for '
                                                        'store '
                                                        'dashboard '
                                                        'tables.',
                                         'kind': 'string',
                                         'name': 'dataset',
                                         'order': 1}}}},
  { 'dcm': { 'auth': 'user',
             'delete': False,
             'out': { 'bigquery': { 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                            'description': 'BigQuery '
                                                                           'dataset '
                                                                           'for '
                                                                           'store '
                                                                           'dashboard '
                                                                           'tables.',
                                                            'kind': 'string',
                                                            'name': 'dataset',
                                                            'order': 1}},
                                    'is_incremental_load': False,
                                    'table': 'Floodlight_CM_Report'}},
             'report': { 'account': { 'field': { 'default': '',
                                                 'description': 'Campaign '
                                                                'Manager '
                                                                'Account '
                                                                'Id.',
                                                 'kind': 'string',
                                                 'name': 'cm_account_id',
                                                 'order': 3}},
                         'body': { 'delivery': { 'emailOwner': False},
                                   'fileName': { 'field': { 'default': 'ITP_Audit_Dashboard_Floodlight',
                                                            'description': 'Campaign '
                                                                           'Manager '
                                                                           'Floodlight '
                                                                           'report '
                                                                           'name.',
                                                            'kind': 'string',
                                                            'name': 'floodlight_report_name',
                                                            'order': 8}},
                                   'floodlightCriteria': { 'dateRange': { 'kind': 'dfareporting#dateRange',
                                                                          'relativeDateRange': 'LAST_30_DAYS'},
                                                           'dimensions': [ { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:site'},
                                                                           { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:floodlightAttributionType'},
                                                                           { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:interactionType'},
                                                                           { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:pathType'},
                                                                           { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:browserPlatform'},
                                                                           { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:platformType'},
                                                                           { 'kind': 'dfareporting#sortedDimension',
                                                                             'name': 'dfa:week'}],
                                                           'floodlightConfigId': { 'dimensionName': 'dfa:floodlightConfigId',
                                                                                   'kind': 'dfareporting#dimensionValue',
                                                                                   'matchType': 'EXACT',
                                                                                   'value': { 'field': { 'default': '',
                                                                                                         'description': 'Floodlight '
                                                                                                                        'Configuration '
                                                                                                                        'Id '
                                                                                                                        'for '
                                                                                                                        'the '
                                                                                                                        'Campaign '
                                                                                                                        'Manager '
                                                                                                                        'floodlight '
                                                                                                                        'report.',
                                                                                                         'kind': 'integer',
                                                                                                         'name': 'floodlight_configuration_id',
                                                                                                         'order': 7}}},
                                                           'metricNames': [ 'dfa:activityClickThroughConversions',
                                                                            'dfa:activityViewThroughConversions',
                                                                            'dfa:totalConversions',
                                                                            'dfa:totalConversionsRevenue'],
                                                           'reportProperties': { 'includeUnattributedCookieConversions': True,
                                                                                 'includeUnattributedIPConversions': False}},
                                   'format': 'CSV',
                                   'kind': 'dfareporting#report',
                                   'name': { 'field': { 'default': 'ITP_Audit_Dashboard_Floodlight',
                                                        'description': 'Campaign '
                                                                       'Manager '
                                                                       'Floodlight '
                                                                       'report '
                                                                       'name.',
                                                        'kind': 'string',
                                                        'name': 'floodlight_report_name',
                                                        'order': 8}},
                                   'schedule': { 'active': True,
                                                 'every': 1,
                                                 'expirationDate': '2029-12-10',
                                                 'repeats': 'DAILY',
                                                 'startDate': '2019-09-11'},
                                   'type': 'FLOODLIGHT'}},
             'timeout': 60}},
  { 'dbm': { 'auth': 'user',
             'datastudio': True,
             'out': { 'bigquery': { 'autodetect_schema': True,
                                    'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                            'description': 'BigQuery '
                                                                           'dataset '
                                                                           'for '
                                                                           'store '
                                                                           'dashboard '
                                                                           'tables.',
                                                            'kind': 'string',
                                                            'name': 'dataset',
                                                            'order': 1}},
                                    'is_incremental_load': False,
                                    'table': 'Dv360_Browser_Report_Dirty'}},
             'report': { 'name': { 'field': { 'default': 'ITP_Audit_Browser_Report',
                                              'description': 'DV360 '
                                                             'Browser '
                                                             'report '
                                                             'name.',
                                              'kind': 'string',
                                              'name': 'dv360_report_name',
                                              'order': 8}}}}},
  { 'sheets': { 'auth': 'user',
                'header': True,
                'out': { 'auth': 'user',
                         'bigquery': { 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                               'description': 'BigQuery '
                                                                              'dataset '
                                                                              'for '
                                                                              'store '
                                                                              'dashboard '
                                                                              'tables.',
                                                               'kind': 'string',
                                                               'name': 'dataset',
                                                               'order': 1}},
                                       'table': 'Environment'}},
                'range': 'A:B',
                'sheet': { 'field': { 'default': '',
                                      'description': 'Sheet '
                                                     'URL '
                                                     'for '
                                                     'the '
                                                     'Segments '
                                                     'sheet.',
                                      'kind': 'string',
                                      'name': 'sheet_url',
                                      'order': 2}},
                'tab': 'Enviroment'}},
  { 'sheets': { 'auth': 'user',
                'header': True,
                'out': { 'auth': 'user',
                         'bigquery': { 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                               'description': 'BigQuery '
                                                                              'dataset '
                                                                              'for '
                                                                              'store '
                                                                              'dashboard '
                                                                              'tables.',
                                                               'kind': 'string',
                                                               'name': 'dataset',
                                                               'order': 1}},
                                       'table': 'Browser'}},
                'range': 'A:C',
                'sheet': { 'field': { 'default': '',
                                      'description': 'Sheet '
                                                     'URL '
                                                     'for '
                                                     'the '
                                                     'Segments '
                                                     'sheet.',
                                      'kind': 'string',
                                      'name': 'sheet_url',
                                      'order': 2}},
                'tab': 'Browser'}},
  { 'sheets': { 'auth': 'user',
                'header': True,
                'out': { 'auth': 'user',
                         'bigquery': { 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                               'description': 'BigQuery '
                                                                              'dataset '
                                                                              'for '
                                                                              'store '
                                                                              'dashboard '
                                                                              'tables.',
                                                               'kind': 'string',
                                                               'name': 'dataset',
                                                               'order': 1}},
                                       'table': 'CM_Browser_lookup'}},
                'range': 'A:C',
                'sheet': { 'field': { 'default': '',
                                      'description': 'Sheet '
                                                     'URL '
                                                     'for '
                                                     'the '
                                                     'Segments '
                                                     'sheet.',
                                      'kind': 'string',
                                      'name': 'sheet_url',
                                      'order': 2}},
                'tab': 'CM_Site_Segments'}},
  { 'sheets': { 'auth': 'user',
                'header': True,
                'out': { 'auth': 'user',
                         'bigquery': { 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                               'description': 'BigQuery '
                                                                              'dataset '
                                                                              'for '
                                                                              'store '
                                                                              'dashboard '
                                                                              'tables.',
                                                               'kind': 'string',
                                                               'name': 'dataset',
                                                               'order': 1}},
                                       'table': 'Device_Type'}},
                'range': 'A:B',
                'sheet': { 'field': { 'default': '',
                                      'description': 'Sheet '
                                                     'URL '
                                                     'for '
                                                     'the '
                                                     'Segments '
                                                     'sheet.',
                                      'kind': 'string',
                                      'name': 'sheet_url',
                                      'order': 2}},
                'tab': 'Device_Type'}},
  { 'sheets': { 'auth': 'user',
                'header': True,
                'out': { 'auth': 'user',
                         'bigquery': { 'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                                               'description': 'BigQuery '
                                                                              'dataset '
                                                                              'for '
                                                                              'store '
                                                                              'dashboard '
                                                                              'tables.',
                                                               'kind': 'string',
                                                               'name': 'dataset',
                                                               'order': 1}},
                                       'table': 'Floodlight_Attribution'}},
                'range': 'A:B',
                'sheet': { 'field': { 'default': '',
                                      'description': 'Sheet '
                                                     'URL '
                                                     'for '
                                                     'the '
                                                     'Segments '
                                                     'sheet.',
                                      'kind': 'string',
                                      'name': 'sheet_url',
                                      'order': 2}},
                'tab': 'Floodlight_Attribution'}},
  { 'itp_audit': { 'account': { 'field': { 'default': '',
                                           'description': 'Campaign '
                                                          'Manager '
                                                          'Account '
                                                          'Id.',
                                           'kind': 'string',
                                           'name': 'cm_account_id',
                                           'order': 3}},
                   'advertiser_ids': { 'field': { 'default': '',
                                                  'description': 'Comma '
                                                                 'separated '
                                                                 'list '
                                                                 'of '
                                                                 'Campaign '
                                                                 'Manager '
                                                                 'Advertiser '
                                                                 'Ids.  '
                                                                 'Leave '
                                                                 'blank '
                                                                 'for '
                                                                 'no '
                                                                 'advertiser '
                                                                 'filtering.',
                                                  'kind': 'string',
                                                  'name': 'advertiser_ids',
                                                  'order': 5}},
                   'auth': 'user',
                   'cm_browser_report_name': { 'field': { 'default': 'ITP_Audit_Dashboard_Browser',
                                                          'description': 'Name '
                                                                         'of '
                                                                         'the '
                                                                         'Campaign '
                                                                         'Manager '
                                                                         'browser '
                                                                         'report.',
                                                          'kind': 'string',
                                                          'name': 'cm_browser_report_name',
                                                          'order': 9}},
                   'dataset': { 'field': { 'default': 'ITP_Audit_Dashboard',
                                           'description': 'BigQuery '
                                                          'dataset '
                                                          'for '
                                                          'store '
                                                          'dashboard '
                                                          'tables.',
                                           'kind': 'string',
                                           'name': 'dataset',
                                           'order': 1}},
                   'sheet': { 'field': { 'default': '',
                                         'description': 'Sheet '
                                                        'URL '
                                                        'for '
                                                        'the '
                                                        'Segments '
                                                        'sheet.',
                                         'kind': 'string',
                                         'name': 'sheet_url',
                                         'order': 2}},
                   'timeout': 60}}]

json_set_fields(TASKS, FIELDS)
project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True)
project.execute()
